In [1]:
import os
import pathlib
import torch
import spacy
from OIE.datasets.validated_splits.contractions import transform_portuguese_contractions, clean_extraction
from OIE.final.matcher import OIE_Match
import pandas as pd
import numpy as np

In [2]:
torch.cuda.is_available()

True

In [15]:
class ArgsRel3_1:
    def __init__(self):
        self.provavel_rel = []
        self.alinhamentos = []
        self.matcher = OIE_Match()
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")

    def get_args_rel(self, ext, sent):
        self.alinhamentos = []
        pos = []
        ext_list = ext.split(" ")
        sent_list = sent.split(" ")
        sent_doc = self.nlp(sent)

        # permutando relações
        # Começa com o maior tamanho de subsequência e vai diminuindo
        for length in range(len(ext_list) - 2, 0, -1):
            for start in range(1, len(ext_list) - length):
                end = start + length
                rel = ext_list[start:end]

                idx = (start, end)
                arg0 = " ".join(ext_list[:idx[0]])
                arg1 = " ".join(ext_list[idx[1]:len(sent_list)])
                rel = " ".join(rel)
                valid = self.matcher.match(sent, arg0, rel, arg1)
                if valid[3]:
                    # colhe pos da relação do alinhamento, o pos usado é o da sent nos tokens da ext
                    aux = []
                    aux_dep = []
                    cur_ext = []
                    cur_dep = []
                    for span in valid[:-1]:
                        span_tk = sent_doc[span[0]:span[1] + 1]
                        for token in span_tk:
                            aux.append(token.pos_)
                            aux_dep.append(token.dep_)
                        cur_ext.append(aux)
                        cur_dep.append(aux_dep)
                        aux = []
                        aux_dep = []
                    pos.append(((arg0, rel, arg1), cur_ext, cur_dep))
                    # utiliza regras no pos da relação para filtrar alinhamentos
                    ali_gerado = ((arg0, rel, arg1), cur_ext, cur_dep)
                    rel_pos = ali_gerado[1][1]
                    rel_dep = ali_gerado[2][1]
                    inicio = [[rel_pos[0], rel_dep[0]]]
                    meio = []
                    for x, y in zip(rel_pos[1:-1], rel_dep[1:-1]):
                        meio.append([x, y])
                    fim = [[rel_pos[-1], rel_dep[-1]]]
                    first = False
                    middle = False
                    middle_counter = 0
                    # inicio
                    for i, tags in enumerate(inicio):
                        p_tag = tags[0]
                        p_dep = tags[1]
                        if p_tag == "ADV" and i == 0 and len(rel_pos) > 1 and rel_pos[1] in ['VERB', 'AUX']:
                            first = True
                            if len(rel_pos) == 2:
                                self.alinhamentos.append(ali_gerado[0])
                                return self.alinhamentos
                        elif p_tag == "ADV" and i == 0 and len(rel_pos) > 1 and rel_pos[1] == 'PRON':
                            first = True
                        elif p_tag == "PRON" and i == 0 and len(rel_pos) > 1 and rel_pos[1] in ['VERB', 'AUX']:
                            first = True
                            if len(rel_pos) == 2:
                                self.alinhamentos.append(ali_gerado[0])
                                return self.alinhamentos
                        elif p_tag == "AUX" and i == 0:
                            first = True
                            if len(rel_pos) == 1:
                                self.alinhamentos.append(ali_gerado[0])
                                return self.alinhamentos
                        elif (p_tag == "VERB" and p_dep == "ROOT") and i == 0:
                            first = True
                            if len(rel_pos) == 1:
                                self.alinhamentos.append(ali_gerado[0])
                                return self.alinhamentos
                        elif p_tag == "VERB" and i == 0:
                            first = True
                            if len(rel_pos) == 1:
                                self.alinhamentos.append(ali_gerado[0])
                                return self.alinhamentos

                    # meio
                    for i, tags in enumerate(meio):
                        p_tag = tags[0]
                        if p_tag in ['ADJ', 'NOUN', 'VERB', "AUX", "DET", "PRON", "SCONJ", "PROPN"] and first:
                            middle_counter += 1
                    if middle_counter == len(meio):
                        middle = True
                    # fim
                    for i, tags in enumerate(fim):
                        p_tag = tags[0]
                        if len(rel_pos) == 2 and p_tag == "VERB" and first:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                        elif len(rel_pos) == 2 and p_tag == "AUX" and first:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                        elif len(rel_pos) == 2 and p_tag == "ADP" and first:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                        elif len(rel_pos) > 2 and p_tag in ["ADP", "VERB", "AUX"] and first and middle:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos

        if len(self.alinhamentos) == 0:
            self.alinhamentos.append((" ", " ", " "))

        return self.alinhamentos

In [20]:
class ArgsRel3_5:
    def __init__(self):
        self.provavel_rel = []
        self.matcher = OIE_Match()
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")
        self.rel_patterns = {
                'VERB',
                'AUX',
                'VERB VERB',
                'VERB AUX',
                'VERB ADP',
                'AUX VERB',
                'ADV VERB',
                'VERB VERB DET',
                'VERB DET ADP',
                'VERB VERB ADP',
                'AUX VERB ADP',
                'PRON VERB ADP',
                'AUX ADV ADP',
                'AUX ADV VERB',
                'VERB DET PROPN ADP',
                'VERB VERB SCONJ VERB',
                'ADV PRON VERB ADP',
                'VERB VERB VERB DET NOUN ADP',
                'VERB VERB SCONJ VERB ADP',
                'VERB VERB DET NOUN ADP',
        }

    def get_args_rel(self, ext, sent):
        alinhamentos = []
        ext_list = ext.split(" ")
        sent_list = sent.split(" ")
        len_ext_list = len(ext_list)
        sent_doc = self.nlp(sent)
        matched_rels = []
        
        #permutando relações
        for i_raw in range(1,len_ext_list):
            for j_raw in range(i_raw+1,len_ext_list):
                if i_raw == len_ext_list-1:
                    rel = ext_list[i_raw]
                else:
                    rel = ext_list[i_raw:j_raw]
                    
                #verificando relações
                idx = (i_raw,j_raw)
                arg0 = " ".join(ext_list[:idx[0]])
                arg1 = " ".join(ext_list[idx[1]:len(sent_list)])
                rel = " ".join(rel)
                
                valid = self.matcher.match(sent, arg0, rel, arg1)
                if valid[3]:
                    matched_rels.append([arg0, rel, arg1, valid])
        
        matched_rels.sort(key = lambda x: len(x[1]), reverse=True)
        
        for matched in matched_rels:                 
            #colhe pos da relação do alinhamento, o pos usado é o da sent nos tokens da ext
            arg0 = matched[0]
            rel = matched[1]
            arg1 = matched[2]
            valid = matched[3]
            
            cur_ext = []
            for span in valid[:-1]:
                aux = ''
                span_tk = sent_doc[span[0]:span[1]+1]
                for token in span_tk:
                    aux += token.pos_ +" "
                cur_ext.append(aux[:-1])
            ali_gerado = ((arg0,rel,arg1),cur_ext)
            
            #busca padrao encontrado nos padroes da lista
            if ali_gerado[1][1] in self.rel_patterns:
                alinhamentos.append(ali_gerado[0])
                return alinhamentos
                
        if len(alinhamentos) == 0:
            alinhamentos.append((" "," "," "))

        return alinhamentos

In [16]:
ali3_1 = ArgsRel3_1()

In [22]:
ext = "Moléculas de ar se movem muito"
sent = "Moléculas de ar se movem muito e colidem com coisas"
ext = transform_portuguese_contractions(ext)
sent = transform_portuguese_contractions(sent)

In [23]:
ali3_1.get_args_rel(ext,ext)

[('Moléculas de ar', 'se movem', 'muito')]

In [24]:
nlp = spacy.load("pt_core_news_lg")

sent2 = nlp(sent)
print("token - pos - dep")
print("-"*50)
for token in sent2:
        print(token.text,"-",token.pos_,"-", token.dep_)
print("------------------------------------------------")
print("sent: ",sent)
print("alignment: ", ali3_1.get_args_rel(ext, sent))


token - pos - dep
--------------------------------------------------
Moléculas - PROPN - nsubj
de - ADP - case
ar - NOUN - nmod
se - PRON - expl
movem - VERB - ROOT
muito - ADV - advmod
e - CCONJ - cc
colidem - VERB - conj
com - ADP - case
coisas - NOUN - obl
------------------------------------------------
sent:  Moléculas de ar se movem muito e colidem com coisas
alignment:  [('Moléculas de ar', 'se movem', 'muito')]


In [19]:
valid = {}
id = 0
with open("development/100-gold.txt", "r", encoding="utf-8") as f:
    lines = f.read()
    samples = lines.split("\n\n")
for ext in samples:
    sent = ""
    arg0 = ""
    rel = ""
    arg1 = ""
    ext_lines = ext.split("\n")
    if ext_lines != ['']:
        for line in ext_lines:
            line = line.split("\t")
            sent += line[0] + " "
            if "ARG0" in line[8]:
                arg0+=line[0] + " "
            if "V" in line[8]:
                rel+=line[0] + " "
            if "ARG1" in line[8]:
                arg1+=line[0] + " "
        valid[id] = {"ext": arg0+" "+rel+" "+arg1,"sent": sent, "arg1": transform_portuguese_contractions(arg0), "rel": transform_portuguese_contractions(rel), "arg2": transform_portuguese_contractions(arg1)}
        id+=1

In [20]:
valid

{0: {'ext': 'um casal  poderia abrir  quatro contas Tesco ',
  'sent': 'Teoricamente , um casal poderia abrir quatro contas Tesco e ganhar 3 % em £ 12,000 – £ 360 . ',
  'arg1': 'um casal',
  'rel': 'poderia abrir',
  'arg2': 'quatro contas Tesco'},
 1: {'ext': 'isso  é  cada vez mais digno de atenção ',
  'sent': 'Um avanço rápido para 2016 e isso é cada vez mais digno de atenção . ',
  'arg1': 'isso',
  'rel': 'é',
  'arg2': 'cada vez mais digno de atenção'},
 2: {'ext': 'as nuvens de chuvas ácidas  podem ter  um lado positivo ',
  'sent': 'Surpreendentemente como pode parecer , até as nuvens de chuvas ácidas podem ter um lado positivo . ',
  'arg1': 'as nuvens de chuvas ácidas',
  'rel': 'podem ter',
  'arg2': 'um lado positivo'},
 3: {'ext': 'O mundo  pode estar  absurdo ',
  'sent': 'O mundo pode estar enfurecido e absurdo - em o entanto por o menos alguém tem dignidade para continuar a protestar contra esse fato . ',
  'arg1': 'O mundo',
  'rel': 'pode estar',
  'arg2': 'absurdo'

In [21]:
arg2 = ali3_1
matcher = OIE_Match()
nlp = spacy.load("pt_core_news_lg")
correct = 0
incorrect = 0
correct_dict = {}
incorrect_dict = {}
json_dict = {}
rel_all = []

for id in valid:
    sentence = valid[id]["sent"]
    sentence = transform_portuguese_contractions(sentence)
    ext = valid[id]["ext"]
    ext = transform_portuguese_contractions(ext)
    print("-"*50)
    print("ext: ",ext)

    ext2 = nlp(ext)
    
    sent_doc = nlp(sentence)
    pos = []
    v = matcher.match(sentence, valid[id]['arg1'], valid[id]['rel'], valid[id]['arg2'])
    if v[3]:
        #colhe pos da relação do alinhamento, o pos usado é o da sent nos tokens da ext
        aux = []
        aux_dep = []
        cur_ext = []
        cur_dep = []
        for span in v[:-1]:
            span_tk = sent_doc[span[0]:span[1]+1]
            pos_dep_ = ''
            for token in span_tk:
                pos_dep_ += token.pos_+'-'+token.dep_+' '
            aux.append(pos_dep_)
            cur_ext.append(aux)
            aux = []
        pos.append(((valid[id]['arg1'],valid[id]['rel'],valid[id]['arg2']),cur_ext, cur_dep))


    alinhamentos = arg2.get_args_rel(ext, sentence)
    print("alinhamentos: ", alinhamentos)
    n_valid = False
    for alinhamento in alinhamentos:
        arg0, rel, arg1 = alinhamento

        arg0 = clean_extraction(arg0)
        rel = clean_extraction(rel)
        arg1 = clean_extraction(arg1)
        try:
            if arg0[-1] == " ":
                arg0 = arg0[:-1]
            if rel[-1] == " ":
                rel = rel[:-1]
            if arg1[-1] == " ":
                arg1 = arg1[:-1]
        except:
            pass

        true_arg0 = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["arg1"])]))
        true_rel = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["rel"])]))
        true_arg1 = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["arg2"])]))
        print("alignment: ", (arg0, rel, arg1))
        if arg0 == true_arg0 and rel == true_rel and arg1 == true_arg1:
            correct_dict[id] = {"ext": ext, "arg0": arg0, "rel": rel, "arg1": arg1}
            json_dict[correct] = {"ID": id, "sent": valid[id]["sent"], "ext": [{
                "arg1": valid[id]["arg1"],
                "rel": valid[id]["rel"],
                "arg2": valid[id]["arg2"]
            }]}
            correct += 1
            n_valid = True
    if not n_valid:
        incorrect += 1
        incorrect_dict[id] = {"ext": ext, "alignment":{"arg0": arg0, "rel": rel, "arg1": arg1}, "expected": {"arg0": true_arg0, "rel": true_rel, "arg1": true_arg1}}
    print("expected: ", (valid[id]["arg1"], valid[id]["rel"], valid[id]["arg2"]))
    print("expected_pos_dep: ", pos[0][1])
    #print("expected_dep: ", pos[0][2])
    print("total: ", correct+incorrect)
    print("correct: ", correct)
    print("incorrect: ", incorrect)

--------------------------------------------------
ext:  um casal poderia abrir quatro contas Tesco
alinhamentos:  [('um casal', 'poderia abrir', 'quatro contas Tesco')]
alignment:  ('um casal', 'poderia abrir', 'quatro contas Tesco')
expected:  ('um casal', 'poderia abrir', 'quatro contas Tesco')
expected_pos_dep:  [['DET-det NOUN-nsubj '], ['VERB-ROOT VERB-xcomp '], ['NUM-nummod NOUN-obj PROPN-appos ']]
total:  1
correct:  1
incorrect:  0
--------------------------------------------------
ext:  isso é cada vez mais digno de atenção
alinhamentos:  [('isso', 'é cada vez', 'mais digno de atenção')]
alignment:  ('isso', 'é cada vez', 'mais digno de atenção')
expected:  ('isso', 'é', 'cada vez mais digno de atenção')
expected_pos_dep:  [['PRON-nsubj '], ['AUX-cop '], ['PROPN-nmod ADP-case ADV-nmod ADJ-conj ADP-case NOUN-obl ']]
total:  2
correct:  1
incorrect:  1
--------------------------------------------------
ext:  as nuvens de chuvas ácidas podem ter um lado positivo
alinhamentos:  [